<a href="https://colab.research.google.com/github/khodid/2020_SAI_MONING2/blob/master/CNNLec10_6_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet

[이론영상- 모딥 시즌1](https://youtu.be/KbNbWTnlYXs?t=393)

Residual Net. 2015년에 발표된 모델.

3.6%대로 Error Rate을 줄였던 굉장한 모델이다. 사람이 판단할 때의 오류율(5%)보다도 낮다.

당시 수많은 대회를 휩쓸었다고.

Alex net의 경우 레이어가 8개고, VGG는 19개까지도 쓴다. 그런데 ResNet은 layer를 152개나 사용한다.

이 많은 학습량을 어떻게 감당할까?

![Resnet](https://miro.medium.com/max/1400/1*G8e3wym0Rs1yPcp62yBgaQ.png)

Resnet에서는 특이하게도 데이터를 몇 개 레이어를 뛰어넘도록 만든다.

그러므로 전체적인 레이어는 깊지만, 실제 학습시에는 그렇게 깊은 레이어를 다루지 않는 것처럼 만드는 것이다.

![Resnet](https://miro.medium.com/max/1984/1*OFfO8VzLv8GNFNRKafvB7w.png)

전체적으로 보면 이렇다.

굉장히 어렵다. 그리고 이 것이 왜 성능이 좋은지도 이해하기 어렵다.



## 코드

이번에도 역시 [torchvision 참고문서](https://pytorch.org/docs/stable/torchvision/models.html)에서 코드를 퍼온다.

SOURCE를 선택하면 뜨는 [이 페이지](https://pytorch.org/docs/stable/_modules/torchvision/models/resnet.html#resnet18)에서 원본 코드를 가져올 수 있다.



In [0]:
import torch
import torch.nn as nn
# from .utils import load_state_dict_from_url
import torch.utils.model_zoo as model_zoo

In [0]:
# 기본적으로 torchvision 문서에서 긁어온 코드지만.. 
# 강의에서 없는 부분은 주석 처리를 하도록 하겠다.
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 
          #  'resnext50_32x4d', 'resnext101_32x8d',
          #  'wide_resnet50_2', 'wide_resnet101_2'
           ]


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    # 'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    # 'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    # 'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    # 'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}

convolution과 관련한 선언이다.

kernel size가 3x3이기 때문에 padding을 1로 해준다고 한다.

In [0]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)
    
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

**Basic Block 부분**은 아까 봤던 그림을 토대로 작성하면 된다.

weight layer를 통과시키고, ReLU를 통과하고, 이걸 또 weight layer 사이로 통과시키고 그 출력을 입력과 더해주면 된다.

![Resnet](https://miro.medium.com/max/1400/1*G8e3wym0Rs1yPcp62yBgaQ.png)

뭔가 피드백 구조에서 피드백 라인만 방향을 바꿔놓은, 그런 구조인데...

다른 분야에도 수학적으로 비슷한 모델을 사용하는 원리가 있을지 궁금하다.

In [0]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        # 가정: size of x = 3x64x64
        identity = x        # identity로 처음 입력 저장, identity 3x64x64

        out = self.conv1(x) # stride가 2라면 3x32x32
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)     
        # identity와 out이 서로 사이즈가 안 맞아서 연산이 불가능할 경우를 커버해준다고 한다. 
        # 솔직히 이해는 못 했다..

        out += identity     
        out = self.relu(out)

        return out

Bottleneck Block 부분

![Bottleneck](https://miro.medium.com/max/2000/1*f7C6lhx50ol9oYifAOu5vw.png)

위 그림에서 오른쪽이 Bottleneck Block 부분이다.


In [0]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        # size 조정을 위한 downsample
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

**Resnet** 부분

In [0]:
class ResNet(nn.Module):
  ## 공부 과정이니
  ## model = _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
  ##                **kwargs)  
  ## model = _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
  ##                 **kwargs)
  ## 인 것을 적어두고 분석을 진행하자.
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        ## 여기의 파라미터들은 각자 알아서 여러가지 시도해보는 걸 추천한다고 함.
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        
        ## inputs  = 64x224x224
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        
        ## outputs = self.conv1(inputs)로 넣는다면 outputs의 shape는 64 x 112 x 112 가 될 것임.
        self.bn1 = norm_layer(self.inplanes)
        ## output shape:  64 x 56 x 56

        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
      
      ## layer 1의 파라미터를 기준으로 어떤 구조인지 분석해보도록 하자.
      ## layer 1에선 self.inplanes = 64 였고
      ## self.layer1 = self._make_layer(Bottleneck, 64, 3)
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        ## self inpulane = 64, planes = 64, bottleneck의 block.expansion은 4다. 따라서 downsample을 써야 한다.
        ## 채널을 맞추기 위해서도 쓰는 편이다.
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride), #(64, 64*4,stride=1)
                norm_layer(planes * block.expansion),                     # 2D 256
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        ## layers.append(block(64, 64, 1, downsamples는 바로 위에 있었음 , self.groups,
        ##                     self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion  ## 64*4로 업데이트됨. 위에서 append 해줬기 때문.
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))
        ##  위의 for 반복문을 통해 아래의 결과가 생성된다.
        ## self.layer1 = [
        ##                Bottleneck(64, 64, 1, downsample)
        ##                Bottleneck(256, 64, 1)
        ##                Bottleneck(256, 64, 1)
        ## ]

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

    

실제로 이 ResNet 부분이 네트워크를 어떻게 만드는 지는 밑에서 확인해보겠다.

In [0]:
def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)                


In [0]:
def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)

위에 주석으로 계산한 크기대로 나오는지 한 번 확인해보자.


In [22]:
res = resnet50()
res

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

layer 1에 대해 계산한 것과 동일하게 Bottleneck은 0, 1, 2로 반복된다.

모델을 공부하고 분석할 때는 이런 식으로 진행하면 된다. 

ResNet에서 처음 등장하는 **"downsample"**은 stride가 2일 때 feature 크기가 줄어드니깐 identity와 크기를 맞춰주기 위해서 사용하는데, 채널 size를 맞추기 위해서도 사용된다고 이해하면 된다고 한다.